In [ ]:
import plotly.express as px
import polars as pl

from fryer import all as fryer

In [ ]:
df_postcode = fryer.data.uk_gov_ons_postcode_directory.read().select(
    pl.col("postcode"),
    pl.col("longitude"),
    pl.col("latitude"),
)
df_postcode.tail().collect()

In [ ]:
df = (
    fryer.data.uk_gov_hm_land_registry_price_paid.read()
    .with_columns(
        month=pl.col("date").dt.month_start(),
        year=pl.col("date").dt.strftime("%Y-01-01").str.to_date("%Y-%m-%d"),
        postcode=pl.col("postcode"),
    )
    .join(df_postcode, on="postcode", how="left")
)

df.tail().collect()

In [ ]:
# Postcodes where the join has failed
df.filter(pl.col("longitude").is_null()).collect()["postcode"].unique()

In [ ]:
for col in ("year", "month"):
    df_len = df.group_by(col).len().collect()
    display(df_len.pipe(px.bar, x=col, y="len", title=col))

In [ ]:
column_x = "year"
for col in (
    "property_type",
    "old_new",
    "tenure_duration",
    "transaction_type",
    "record_status_monthly_file_only",
):
    df_len = df.group_by([column_x, col]).len().collect()
    display(
        df_len.pipe(
            px.bar,
            x=column_x,
            y="len",
            color=col,
            title=col,
            category_orders={
                col: df_len.sort(by="len", descending=True)[col].to_list()
            },
        )
    )

In [ ]:
# Note, this does not necessarily tell us if the prices are increasing/decreasing due to houses being sold not being the same in time

column_group_by = "month"
column_value = "price"
map_quantiles = {
    f"{column_value}_q{quantile:.2f}": quantile
    for quantile in [
        # 0.0,
        # 0.01,
        0.05,
        0.1,
        0.25,
        0.5,
        0.75,
        0.9,
        0.95,
        # 0.99,
        # 1.0,
    ]
}
columns_quantiles = list(map_quantiles.keys())

(
    df.group_by(column_group_by)
    .agg(
        **{
            column_quantile: pl.col(column_value).quantile(quantile)
            for column_quantile, quantile in map_quantiles.items()
        }
    )
    .sort(by=column_group_by)
    .collect()
    .pipe(px.line, x=column_group_by, y=columns_quantiles, log_y=True)
)

In [ ]:
# Note, this does not necessarily tell us if the prices are increasing/decreasing due to houses being sold not being the same in time

columns_group_by = ["month", "property_type"]
column_value = "price"
map_quantiles = {
    f"{column_value}_q{quantile:.2f}": quantile
    for quantile in [
        # 0.0,
        # 0.01,
        0.05,
        0.1,
        0.25,
        0.5,
        0.75,
        0.9,
        0.95,
        # 0.99,
        # 1.0,
    ]
}
columns_quantiles = list(map_quantiles.keys())

(
    df.group_by(columns_group_by)
    .agg(
        **{
            column_quantile: pl.col(column_value).quantile(quantile)
            for column_quantile, quantile in map_quantiles.items()
        }
    )
    .sort(by=columns_group_by)
    .collect()
    .pipe(
        px.line,
        x=columns_group_by[0],
        y=columns_quantiles,
        log_y=True,
        facet_row=columns_group_by[1],
        height=1_600,
    )
    .update_yaxes(matches=None)
)

In [ ]:
# Note, this does not necessarily tell us if the prices are increasing/decreasing due to houses being sold not being the same in time

columns_group_by = ["month", "tenure_duration"]
column_value = "price"
map_quantiles = {
    f"{column_value}_q{quantile:.2f}": quantile
    for quantile in [
        # 0.0,
        # 0.01,
        0.05,
        0.1,
        0.25,
        0.5,
        0.75,
        0.9,
        0.95,
        # 0.99,
        # 1.0,
    ]
}
columns_quantiles = list(map_quantiles.keys())

(
    df.group_by(columns_group_by)
    .agg(
        **{
            column_quantile: pl.col(column_value).quantile(quantile)
            for column_quantile, quantile in map_quantiles.items()
        }
    )
    .sort(by=columns_group_by)
    .collect()
    .pipe(
        px.line,
        x=columns_group_by[0],
        y=columns_quantiles,
        log_y=True,
        facet_row=columns_group_by[1],
        height=1_600,
    )
    .update_yaxes(matches=None)
)

In [ ]:
(
    df.filter(pl.col("county") == "GREATER LONDON")
    .group_by(["year", "longitude", "latitude"])
    .agg(price=pl.col("price").median())
    .collect()
    .pipe(
        px.scatter_map,
        lat="latitude",
        lon="longitude",
        color="price",
        opacity=0.25,
        animation_frame="year",
    )
)